In [1]:
import os
from pathlib import Path
import re
from itertools import zip_longest, cycle


FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day22.txt'

# Part One


In [2]:

with open(FOLDER / in_file) as f:
    m, data = f.read().split('\n\n')
    instructions = re.findall(r'\d+|[LR]', data)
    m = [list(l) for l in m.split('\n')]


class OverflowError(Exception): pass    
class UnderflowError(Exception): pass    
class BlockedError(Exception): pass


class Line:
    def __init__(self, start_index, end_index, axis):
        self.start = start_index
        self.end = end_index
        self.block_indexes = set() # relative to zero
        self.axis = axis

    def __repr__(self):
        return ' ' * self.start + ''.join('#' if i in self.block_indexes else '.' for i in range(self.start, self.end))
    
    def isblocked(self, i):
        return i in self.block_indexes
    
    def move(self, start_index, d):
        '''move backward and forward one. Raise when moving is not possible'''
        
        start_index =  start_index + d

        if start_index in self.block_indexes:
            raise BlockedError()
        
        if start_index >= self.end:
            raise OverflowError()
        
        elif start_index < self.start:
            raise UnderflowError()
        
        return {self.axis: start_index}

    
    
class Map:
    directions = (
        ('>',  1, 'x', 'y'), # icon, axis
        ('v',  1, 'y', 'x'),
        ('<', -1, 'x', 'y'),
        ('^', -1, 'y', 'x')
    )
    
    def __init__(self, map_rows):
        self.rows = self.make_lines(map_rows, 'x')
        self.cols = self.make_lines(zip_longest(*(m), fillvalue=' '), 'y')
        self.lines = {'x': self.rows, 'y': self.cols}

        #self.current_line_group = next(self.lines)
        self.position = {
            'd': 0, # index into directions list
            'x': self.rows[0].start,
            'y': 0
        }
        
    def turn(self, instruction):
        if instruction == 'R':
            self.position['d'] = (self.position['d'] + 1) % 4
        else: 
            self.position['d'] = (self.position['d'] - 1) % 4
        
    def move(self, amount):
        #position = self.position
        direction = self.directions[self.position['d']]
        current_line_group = self.lines[direction[2]]
        line = current_line_group[self.position[direction[3]]]
        for _ in range(amount):    
            try:
                self.position.update(line.move(self.position[direction[2]], direction[1]))
            except BlockedError:
                break
            except OverflowError:
                self.position[direction[2]] = line.start
            except UnderflowError:
                self.position[direction[2]] = line.end - 1
                    
    def score(self):
        y = 1 + self.position['y']
        x = 1 + self.position['x']
        direction_score = self.position['d']
        
        return 1000 * y  + 4 * x + direction_score

            
    @staticmethod
    def make_lines(m, axis):
        rows = []
        for line in m:
            i = next(i for i, c in enumerate(line) if c in '.#')
            row = Line(i, i, axis)
            for idx, c in enumerate(line[i:], i):           
                if c == '#':
                    row.block_indexes.add(idx)
                if c not in '#.':
                    break
                row.end += 1

            rows.append(row)
        return rows


In [3]:

the_map = Map(m)
for i in instructions:
    if i in ['R', 'L']:
        the_map.turn(i)
    else:
        the_map.move(int(i))


the_map.position, the_map.score()

({'d': 2, 'x': 61, 'y': 148}, 149250)

# Part 2

This has **got** to be the hard way to do this.  
Probably better to work out the 3D vectors and math. :yuck:

In [6]:
CUBE_SIZE = 50

class CubeMap(Map):
    def move(self, amount):
        direction = self.directions[self.position['d']]
        current_line_group = self.lines[direction[2]]
        select_index = self.position[direction[3]]

        line = current_line_group[self.position[direction[3]]]
        
        for _ in range(amount):
            direction = self.directions[self.position['d']]
            motion_axis = direction[2]
            try:
                self.position.update(line.move(self.position[motion_axis], direction[1]))
            except BlockedError:
                break
            except OverflowError:
                side_index = select_index // CUBE_SIZE
                match (motion_axis, side_index):
                    case ['x', 0]: # E -> S
                        d = 2
                        y = CUBE_SIZE * 3 - self.position['y'] - 1 
                        line = self.rows[y]
                        x = line.end - 1
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}
                    case ['x', 1]: # B -> E 
                        d = 3
                        x = self.position['y'] + CUBE_SIZE 
                        line = self.cols[x]
                        y = line.end - 1                    
                        if line.isblocked(y):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}                    

                    case ['x', 2]: # S -> E 
                        d = 2
                        y = CUBE_SIZE * 3 - 1 - self.position['y']
                        line = self.rows[y]
                        x = line.end - 1
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}
                        
                    case ['x', 3]: # T -> S 
                        d = 3
                        x = self.position['y'] - CUBE_SIZE * 2
                        line = self.cols[x]
                        y = line.end - 1
                        if line.isblocked(y):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}
                        
                    case ['y', 0]: # T -> E
                        d = 1
                        x = self.position['x'] + CUBE_SIZE * 2
                        line = self.cols[x]
                        y = line.start
                        if line.isblocked(y):
                            break                        
                        self.position = {'d': d, 'x': x, 'y': y}
           
                    case ['y', 1]: # S -> T
                        d = 2
                        y = self.position['x'] + CUBE_SIZE * 2
                        line = self.rows[y]
                        x = line.end - 1
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}                                   
                    case['y', 2]: # E -> B
                        d = 2
                        y = self.position['x'] - CUBE_SIZE
                        line = self.rows[y]
                        x = line.end - 1
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}   
                        
            except UnderflowError:
                side_index = select_index // CUBE_SIZE
                match (motion_axis, side_index):
                    case ['x', 0]: # N -> W
                        d = 0
                        y = CUBE_SIZE * 3 - self.position['y'] - 1
                        line = self.rows[y]
                        x = line.start
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}  
                    case ['x', 1]: # B -> W
                        d = 1
                        x = self.position['y'] - CUBE_SIZE
                        line = self.cols[x]
                        y = line.start
                        if line.isblocked(y):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}         
                    case ['x', 2]: # W -> N
                        d = 0
                        y = CUBE_SIZE * 3 - self.position['y'] - 1
                        line = self.rows[y]
                        x = line.start
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}                      
                    case ['x', 3]: # T -> N
                        d = 1
                        x = self.position['y'] - CUBE_SIZE * 2
                        line = self.cols[x] 
                        y = line.start
                        if line.isblocked(y):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}     
                    case ['y', 0]: # W -> B
                        d = 0
                        y = self.position['x'] + CUBE_SIZE
                        line = self.rows[y]
                        x = line.start
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}                         
                    case ['y', 1]: # N -> T
                        d = 0
                        y = self.position['x'] + CUBE_SIZE * 2
                        line = self.rows[y]
                        x = line.start
                        if line.isblocked(x):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}
      
                    case ['y', 2]: # E -> T
                        d = 3
                        x = self.position['x'] - CUBE_SIZE * 2
                        line = self.cols[x]
                        y = line.end - 1
                        if line.isblocked(y):
                            break
                        self.position = {'d': d, 'x': x, 'y': y}    

the_map = CubeMap(m)

for i in instructions:
    if i in ['R', 'L']:
        the_map.turn(i)
    else:
        the_map.move(int(i))


the_map.position, the_map.score()

({'d': 2, 'x': 114, 'y': 11}, 12462)

In [5]:
the_map = CubeMap(m)

# E -> S
the_map.position = {'d': 0, 'x': 149, 'y': 0}
the_map.move(1)
assert the_map.position == {'d': 2, 'y': 149, 'x':99}

# B -> E
the_map.position = {'d': 0, 'x': 99, 'y': 51}
the_map.move(1)
assert the_map.position == {'d': 3, 'x': 101, 'y':49}

# S -> E
the_map.position = {'d': 0, 'x': 99, 'y': 101}
the_map.move(1)
assert the_map.position == {'d': 2, 'x': 149, 'y':48}

# T -> S
the_map.position = {'d': 0, 'x': 49, 'y': 152}
the_map.move(1)
assert the_map.position == {'d': 3, 'x': 52, 'y':149}

# T -> E
the_map.position = {'d': 1, 'x': 3, 'y': 199}
the_map.move(1)
assert the_map.position == {'d': 1, 'x': 103, 'y':0}

# S -> T
the_map.position = {'d': 1, 'x': 99, 'y': 149}
the_map.move(1)
assert the_map.position == {'d': 2, 'x': 49, 'y':199}

# E -> B
the_map.position = {'d': 1, 'x': 149, 'y': 49}
the_map.move(1)
assert the_map.position == {'d': 2, 'x': 99, 'y':99}

# Under
# N -> W
the_map.position = {'d': 2, 'x': 50, 'y': 0}
the_map.move(1)
assert the_map.position == {'d': 0, 'x': 0, 'y':149}

# B -> W
the_map.position = {'d': 2, 'x': 50, 'y': 52}
the_map.move(1)
assert the_map.position == {'d': 1, 'x': 2, 'y':100}

# W -> N
the_map.position = {'d': 2, 'x': 0, 'y': 149}
the_map.move(1)
assert the_map.position == {'d': 0, 'x': 50, 'y':0}

# T -> N
the_map.position = {'d': 2, 'x': 0, 'y': 152}
the_map.move(1)
assert the_map.position == {'d': 1, 'x': 52, 'y':0}

# W -> B
the_map.position = {'d': 3, 'x': 2, 'y': 100}
the_map.move(1)
assert the_map.position == {'d': 0, 'x': 50, 'y':52}

# N -> T
the_map.position = {'d': 3, 'x': 50, 'y': 0}
the_map.move(1)
assert the_map.position == {'d': 0, 'x': 0, 'y':150}

# E -> T
the_map.position = {'d': 3, 'x': 101, 'y': 0}
the_map.move(1)
assert the_map.position == {'d': 3, 'x': 1, 'y':199}


# More than one
# E -> B hit block
the_map.position = {'d': 1, 'x': 101, 'y': 49}
the_map.move(10)
assert the_map.position == {'d': 2, 'x': 97, 'y':51}

the_map.position = {'d': 3, 'x': 36, 'y': 152}
the_map.move(53)
assert the_map.position == {'d': 3, 'x': 36, 'y':100}
